Terminal command to fetch SST data: `wget -r -l1 -nc -t 50 https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/netcdf/`.

This downloads all SST datasets in NOAA's directory (indexed by year and month from January 1854 to April 2024).

In [2]:
import xarray as xr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
file_pattern = "ersst.v5.{year:04d}{month:02d}.nc"

start_year = 1854
end_year = 2024

yearly_means = []

for year in range(start_year, end_year + 1):
    monthly_data = []
    
    if year == 2024:
        for month in range(1, 5):
            file_path = file_pattern.format(year=year, month=month)
            ds = xr.open_dataset(file_path)
            monthly_data.append(ds['ssta'])
            ds.close()
    else:
        for month in range(1, 13):
            file_path = file_pattern.format(year=year, month=month)
            ds = xr.open_dataset(file_path)
            monthly_data.append(ds['ssta'])
            ds.close()
        
    yearly_ds = xr.concat(monthly_data, dim='time')
    yearly_mean = yearly_ds.mean(dim='time')
    yearly_means.append(yearly_mean)

yearly_means_ds = xr.concat(yearly_means, dim='year')

output_file = "annual_means.nc"
yearly_means_ds.to_netcdf(output_file)

In [56]:
ds0 = xr.open_dataset("annual_means.nc")

In [76]:
yearly_means_ds = xr.open_dataset("annual_means.nc")

start_year = 1854
end_year = 2024
decade_means = []

for decade_start in range(start_year, end_year, 10):
    decade_end = decade_start + 9
    decade_data = yearly_means_ds.sel(year=slice(decade_start - 1854, decade_end - 1854))
    decade_mean = decade_data.mean(dim='year') 
    decade_means.append(decade_mean)

last_decade_data = yearly_means_ds.sel(year=slice(2020 - 1854, 2024 - 1854))
last_decade_mean = last_decade_data.mean(dim='year')
decade_means.append(last_decade_mean)

decade_means_ds = xr.concat(decade_means, dim='decade')
decade_means_ds.coords['lon'] = (decade_means_ds.coords['lon'] + 180) % 360 - 180
decade_means_ds = decade_means_ds.sortby(decade_means_ds.lon)

decade_means_ds = decade_means_ds.reindex(lat=decade_means_ds['lat'][::-1])

output_file = "decade_means.nc"

decade_means_ds.to_netcdf(output_file)

In [77]:
ds = xr.open_dataset("decade_means.nc")

In [78]:
ds

<xarray.Dataset>
Dimensions:  (lat: 89, lev: 1, lon: 180, decade: 18)
Coordinates:
  * lat      (lat) float64 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lev      (lev) float64 0.0
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
Dimensions without coordinates: decade
Data variables:
    ssta     (decade, lev, lat, lon) float32 ...

In [84]:
latitude_bounds = [-14.249902857461892, 48.232259677999046]
longitude_bounds = [-107.03818115217715, -13.070319058558587]

In [85]:
cropped_data = ds.sel(lat=slice(latitude_bounds[1], latitude_bounds[0]),
                                    lon=slice(longitude_bounds[0], longitude_bounds[1]))

cropped_data = cropped_data.reset_index('lev', drop=True)

cropped_data.to_netcdf("cropped_sst_decade.nc")
cropped_data

<xarray.Dataset>
Dimensions:  (lat: 32, lon: 47, decade: 18, lev: 1)
Coordinates:
  * lat      (lat) float64 48.0 46.0 44.0 42.0 40.0 ... -8.0 -10.0 -12.0 -14.0
  * lon      (lon) float64 -106.0 -104.0 -102.0 -100.0 ... -18.0 -16.0 -14.0
Dimensions without coordinates: decade, lev
Data variables:
    ssta     (decade, lev, lat, lon) float32 ...

In [66]:
cropped_data['ssta'].sel(lon = 180, lat = 0, decade = 1).to_dict("test.json")

{'dims': ('lev',),
 'attrs': {},
 'data': [-0.2639959454536438],
 'coords': {'lat': {'dims': (),
   'attrs': {'units': 'degrees_north',
    'long_name': 'Latitude',
    'standard_name': 'latitude',
    'axis': 'Y',
    'comment': 'Uniform grid from -88 to 88 by 2'},
   'data': 0.0},
  'lon': {'dims': (),
   'attrs': {'units': 'degrees_east',
    'long_name': 'Longitude',
    'standard_name': 'longitude',
    'axis': 'X',
    'comment': 'Uniform grid from 0 to 358 by 2'},
   'data': 180.0}},
 'name': 'ssta'}

In [67]:
cropped_data

<xarray.Dataset>
Dimensions:  (lat: 33, lon: 37, decade: 18, lev: 1)
Coordinates:
  * lat      (lat) float64 -32.0 -30.0 -28.0 -26.0 -24.0 ... 26.0 28.0 30.0 32.0
  * lon      (lon) float64 144.0 146.0 148.0 150.0 ... 210.0 212.0 214.0 216.0
Dimensions without coordinates: decade, lev
Data variables:
    ssta     (decade, lev, lat, lon) float32 ...

In [68]:
ds0['ssta'].sel(year=12).to_dict("test.json")

{'dims': ('lev', 'lat', 'lon'),
 'attrs': {},
 'data': [[[nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
    nan,
 